In [1]:
import torch
import torch.nn.functional as F
import numpy as np
import pandas as pd
import time
import re
import random
from unidecode import unidecode
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests

In [3]:
def GET_UA():
    uastrings = ["Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.111 Safari/537.36",\
                "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/28.0.1500.72 Safari/537.36",\
                "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10) AppleWebKit/600.1.25 (KHTML, like Gecko) Version/8.0 Safari/600.1.25",\
                "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:33.0) Gecko/20100101 Firefox/33.0",\
                "Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.111 Safari/537.36",\
                "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.111 Safari/537.36",\
                "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit/600.1.17 (KHTML, like Gecko) Version/7.1 Safari/537.85.10",\
                "Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko",\
                "Mozilla/5.0 (Windows NT 6.3; WOW64; rv:33.0) Gecko/20100101 Firefox/33.0",\
                "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.104 Safari/537.36"\
                ]
 
    return random.choice(uastrings)

In [4]:
def parse_url(url):
 
    headers = {'User-Agent': GET_UA()}
    content = None
 
    try:
        response = requests.get(url, headers=headers)
        ct = response.headers['Content-Type'].lower().strip()
 
        if 'text/html' in ct:
            content = response.content
            soup = BeautifulSoup(content)
        else:
            content = response.content
            soup = None
 
    except Exception as e:
        print('Error:', str(e))
 
    return content, soup, ct

In [5]:
players = pd.read_csv('player_index.csv')
player_id = players['Player-additional'].values.tolist()
player_name = players['Player'].values.tolist()
league_num =len(player_id)
i = 0
for name in player_name:
    player_name[i] = name.replace(" ", "_")
    i = i+1
print(player_name[0:10])
print(len(player_name))

['Jayson_Tatum', 'Joel_Embiid', 'Shai_Gilgeous-Alexander', 'Luka_Dončić', 'Julius_Randle', 'Damian_Lillard', 'Anthony_Edwards', 'Giannis_Antetokounmpo', 'Trae_Young', 'Donovan_Mitchell']
520


In [6]:
base = 'https://www.basketball-reference.com/players/c/bealbr01/gamelog/'
# NBA season we will be analyzing
year = '2023'
# URL page we will scraping (see image above)
url = base+year
print(url)
# this is the HTML from the given URL
html = urlopen(url)
soup = BeautifulSoup(html)

https://www.basketball-reference.com/players/c/bealbr01/gamelog/2023


In [7]:
# use findALL() to get the column headers
soup.findAll('tr', limit=40)
# use getText()to extract the text we need into a list
i = 0
while i < 40:
    headers = [th.getText() for th in soup.findAll('tr', limit=40)[i].findAll('th')]
    i = i +1
    if headers:
        idx = i
        i = 41

headers = headers[1:]
print(headers)

['G', 'Date', 'Age', 'Tm', '\xa0', 'Opp', '\xa0', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'GmSc', '+/-']


In [8]:
# avoid the first header row
rows = soup.findAll('tr')
player_stats = [[td.getText() for td in rows[i].findAll('td')]
            for i in range(len(rows))]
stats = pd.DataFrame(player_stats, columns = headers)
stats

,G,Date,Age,Tm,,Opp,,GS,MP,FG,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-
0,,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,0-9,1,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,10-19,1,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,20-29,8,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,30-39,36,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,,2023-04-04,29-280,WAS,,MIL,L (-12),Inactive,None,None,...,None,None,None,None,None,None,None,None,None,None
116,,2023-04-05,29-281,WAS,@,ATL,L (-18),Inactive,None,None,...,None,None,None,None,None,None,None,None,None,None
117,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
118,,2023-04-07,29-283,WAS,,MIA,W (+6),Inactive,None,None,...,None,None,None,None,None,None,None,None,None,None


In [41]:
def get_stats(num):
    for person in range(num):
        # NBA season we will be analyzing
        year = '2023'
        player = player_id[person]
        person = person
        base = f'https://www.basketball-reference.com/players/c/{player}/gamelog/'
        # URL page we will scraping (see image above)
        url = base+year
        print(url)
        time.sleep(2.95)
        # this is the HTML from the given URL
        content, soup, ct = parse_url(url)
        
        # use findALL() to get the column headers
        soup.findAll('tr', limit=40)
        # use getText()to extract the text we need into a list
        i = 0
        while i < 40:
            headers = [th.getText() for th in soup.findAll('tr', limit=40)[i].findAll('th')]
            i = i +1
            if headers:
                idx = i
                i = 41

        headers = headers[1:]

        rows = soup.findAll('tr')
        player_stats = [[td.getText() for td in rows[i].findAll('td')]
                    for i in range(len(rows))]
        stats = pd.DataFrame(player_stats, columns = headers)
        stats.dropna(inplace=True)
        stats.insert(0, 'Player', player_name[person])
        stats.index = range(len(stats))
        if person == 0:
            data_core = stats
        else:
            data_core = pd.concat([data_core,stats])
        print(person,player_name[person])
    return data_core


In [42]:
test = get_stats(len(player_name))

https://www.basketball-reference.com/players/c/tatumja01/gamelog/2023
0 Jayson_Tatum
https://www.basketball-reference.com/players/c/embiijo01/gamelog/2023
1 Joel_Embiid
https://www.basketball-reference.com/players/c/gilgesh01/gamelog/2023
2 Shai_Gilgeous-Alexander
https://www.basketball-reference.com/players/c/doncilu01/gamelog/2023
3 Luka_Dončić
https://www.basketball-reference.com/players/c/randlju01/gamelog/2023
4 Julius_Randle
https://www.basketball-reference.com/players/c/lillada01/gamelog/2023
5 Damian_Lillard
https://www.basketball-reference.com/players/c/edwaran01/gamelog/2023
6 Anthony_Edwards
https://www.basketball-reference.com/players/c/antetgi01/gamelog/2023
7 Giannis_Antetokounmpo
https://www.basketball-reference.com/players/c/youngtr01/gamelog/2023
8 Trae_Young
https://www.basketball-reference.com/players/c/mitchdo01/gamelog/2023
9 Donovan_Mitchell
https://www.basketball-reference.com/players/c/lavinza01/gamelog/2023
10 Zach_LaVine
https://www.basketball-reference.com/pl

94 Norman_Powell
https://www.basketball-reference.com/players/c/willija06/gamelog/2023
95 Jalen_Williams
https://www.basketball-reference.com/players/c/martike04/gamelog/2023
96 Kenyon_Martin_Jr.
https://www.basketball-reference.com/players/c/brogdma01/gamelog/2023
97 Malcolm_Brogdon
https://www.basketball-reference.com/players/c/whitede01/gamelog/2023
98 Derrick_White
https://www.basketball-reference.com/players/c/smithja05/gamelog/2023
99 Jabari_Smith_Jr.
https://www.basketball-reference.com/players/c/goberru01/gamelog/2023
100 Rudy_Gobert
https://www.basketball-reference.com/players/c/mcdanja02/gamelog/2023
101 Jaden_McDaniels
https://www.basketball-reference.com/players/c/claxtni01/gamelog/2023
102 Nic_Claxton
https://www.basketball-reference.com/players/c/ballla01/gamelog/2023
103 LaMelo_Ball
https://www.basketball-reference.com/players/c/strusma01/gamelog/2023
104 Max_Strus
https://www.basketball-reference.com/players/c/murrake02/gamelog/2023
105 Keegan_Murray
https://www.basketb

188 Moritz_Wagner
https://www.basketball-reference.com/players/c/anderky01/gamelog/2023
189 Kyle_Anderson
https://www.basketball-reference.com/players/c/sharpsh01/gamelog/2023
190 Shaedon_Sharpe
https://www.basketball-reference.com/players/c/alvarjo01/gamelog/2023
191 Jose_Alvarado
https://www.basketball-reference.com/players/c/vincega01/gamelog/2023
192 Gabe_Vincent
https://www.basketball-reference.com/players/c/hylanbo01/gamelog/2023
193 Bones_Hyland
https://www.basketball-reference.com/players/c/joneshe01/gamelog/2023
194 Herbert_Jones
https://www.basketball-reference.com/players/c/thomaca02/gamelog/2023
195 Cam_Thomas
https://www.basketball-reference.com/players/c/greendr01/gamelog/2023
196 Draymond_Green
https://www.basketball-reference.com/players/c/mccontj01/gamelog/2023
197 T.J._McConnell
https://www.basketball-reference.com/players/c/curryse01/gamelog/2023
198 Seth_Curry
https://www.basketball-reference.com/players/c/looneke01/gamelog/2023
199 Kevon_Looney
https://www.basketba

282 Josh_Christopher
https://www.basketball-reference.com/players/c/simmobe01/gamelog/2023
283 Ben_Simmons
https://www.basketball-reference.com/players/c/watanyu01/gamelog/2023
284 Yuta_Watanabe
https://www.basketball-reference.com/players/c/metuch01/gamelog/2023
285 Chimezie_Metu
https://www.basketball-reference.com/players/c/liveris01/gamelog/2023
286 Isaiah_Livers
https://www.basketball-reference.com/players/c/gayru01/gamelog/2023
287 Rudy_Gay
https://www.basketball-reference.com/players/c/wrighde01/gamelog/2023
288 Delon_Wright
https://www.basketball-reference.com/players/c/harremo01/gamelog/2023
289 Montrezl_Harrell
https://www.basketball-reference.com/players/c/tateja01/gamelog/2023
290 Jae'Sean_Tate
https://www.basketball-reference.com/players/c/braunch01/gamelog/2023
291 Christian_Braun
https://www.basketball-reference.com/players/c/cancavl01/gamelog/2023
292 Vlatko_Čančar
https://www.basketball-reference.com/players/c/pokusal01/gamelog/2023
293 Aleksej_Pokusevski
https://www.b

375 Blake_Wesley
https://www.basketball-reference.com/players/c/knighna01/gamelog/2023
376 Nathan_Knight
https://www.basketball-reference.com/players/c/fernabr01/gamelog/2023
377 Bruno_Fernando
https://www.basketball-reference.com/players/c/dunnkr01/gamelog/2023
378 Kris_Dunn
https://www.basketball-reference.com/players/c/schofad01/gamelog/2023
379 Admiral_Schofield
https://www.basketball-reference.com/players/c/chrisma02/gamelog/2023
380 Max_Christie
https://www.basketball-reference.com/players/c/hernaju01/gamelog/2023
381 Juancho_Hernangómez
https://www.basketball-reference.com/players/c/ryanma01/gamelog/2023
382 Matt_Ryan
https://www.basketball-reference.com/players/c/walkeja01/gamelog/2023
383 Jabari_Walker
https://www.basketball-reference.com/players/c/bantoda01/gamelog/2023
384 Dalano_Banton
https://www.basketball-reference.com/players/c/leesa01/gamelog/2023
385 Saben_Lee
https://www.basketball-reference.com/players/c/holmeri01/gamelog/2023
386 Richaun_Holmes
https://www.basketba

469 Tyrese_Martin
https://www.basketball-reference.com/players/c/rhodeja01/gamelog/2023
470 Jared_Rhoden
https://www.basketball-reference.com/players/c/boehebu01/gamelog/2023
471 Buddy_Boeheim
https://www.basketball-reference.com/players/c/pippesc02/gamelog/2023
472 Scotty_Pippen_Jr.
https://www.basketball-reference.com/players/c/lenal01/gamelog/2023
473 Alex_Len
https://www.basketball-reference.com/players/c/quetane01/gamelog/2023
474 Neemias_Queta
https://www.basketball-reference.com/players/c/mobleis01/gamelog/2023
475 Isaiah_Mobley
https://www.basketball-reference.com/players/c/sarrol01/gamelog/2023
476 Olivier_Sarr
https://www.basketball-reference.com/players/c/campafa01/gamelog/2023
477 Facundo_Campazzo
https://www.basketball-reference.com/players/c/harpero02/gamelog/2023
478 Ron_Harper_Jr.
https://www.basketball-reference.com/players/c/willivi01/gamelog/2023
479 Vince_Williams_Jr.
https://www.basketball-reference.com/players/c/brownke03/gamelog/2023
480 Kendall_Brown
https://www

In [43]:
with pd.option_context('display.max_rows', None,
                       'display.precision', 3,
                       ):
    display(test.sample(5))

,Player,G,Date,Age,Tm,,Opp,,GS,MP,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-
54,Malcolm_Brogdon,55,2023-03-06,30-085,BOS,@,CLE,L (-4),0,36:28,...,4,4,5,0,0,1,4,24,19.5,-7
17,Cedi_Osman,18,2022-11-23,27-229,CLE,,POR,W (+18),0,27:07,...,4,4,3,0,0,1,2,3,0.7,+17
19,Shai_Gilgeous-Alexander,20,2022-11-28,24-139,OKC,@,NOP,L (-4),1,35:38,...,3,6,4,2,0,7,4,31,17.5,-9
25,Kevin_Knox,26,2022-12-20,23-131,DET,,UTA,L (-15),0,18:33,...,3,3,1,0,0,1,1,9,8.3,+9
27,James_Bouknight,28,2023-03-26,22-189,CHO,,DAL,W (+6),0,10:03,...,3,3,0,1,1,0,1,5,4.5,-8


In [44]:
test['Player'].unique()

array(['Jayson_Tatum', 'Joel_Embiid', 'Shai_Gilgeous-Alexander',
       'Luka_Dončić', 'Julius_Randle', 'Damian_Lillard',
       'Anthony_Edwards', 'Giannis_Antetokounmpo', 'Trae_Young',
       'Donovan_Mitchell', 'Zach_LaVine', 'DeMar_DeRozan', "De'Aaron_Fox",
       'Jaylen_Brown', 'Lauri_Markkanen', 'Nikola_Jokić', 'Jalen_Brunson',
       'Pascal_Siakam', 'Jordan_Poole', 'Kyrie_Irving', 'Jalen_Green',
       'Ja_Morant', 'Bam_Adebayo', 'Stephen_Curry', 'Kristaps_Porziņģis',
       'Mikal_Bridges', 'LeBron_James', 'Darius_Garland', 'CJ_McCollum',
       'Kyle_Kuzma', 'Dejounte_Murray', 'Franz_Wagner', 'Klay_Thompson',
       'Domantas_Sabonis', 'Paul_George', 'Terry_Rozier',
       'Paolo_Banchero', 'Jimmy_Butler', 'Keldon_Johnson',
       'Anfernee_Simons', 'RJ_Barrett', 'Jerami_Grant',
       'Bojan_Bogdanović', 'Jordan_Clarkson', 'Nikola_Vučević',
       'Spencer_Dinwiddie', 'Kevin_Durant', 'Buddy_Hield', 'Devin_Booker',
       'Tyler_Herro', 'Anthony_Davis', 'Fred_VanVleet', 'Jam

In [45]:
save = test.copy(deep=True)
save.tail()

,Player,G,Date,Age,Tm,,Opp,,GS,MP,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-
2,Marko_Simonovic,3,2023-02-16,23-124,CHI,,MIL,L (-12),0,1:48,...,0,0,0,0,0,0,1,0,-0.4,+8
3,Marko_Simonovic,4,2023-03-31,23-167,CHI,@,CHO,W (+30),0,3:58,...,0,1,0,0,0,0,1,0,-0.4,-1
4,Marko_Simonovic,5,2023-04-04,23-171,CHI,,ATL,L (-18),0,2:06,...,0,0,0,0,0,0,0,2,1.0,0
5,Marko_Simonovic,6,2023-04-05,23-172,CHI,@,MIL,L (-13),0,0:39,...,0,0,0,0,0,0,0,0,0.0,+3
0,Alondes_Williams,1,2022-12-10,23-174,BRK,@,IND,W (+3),0,5:16,...,1,1,0,0,0,2,1,0,-2.1,-5


In [46]:
test['Date'] = pd.to_datetime(test['Date'])
data = test
for i in range(len(data)):
    data.iloc[i,0] = unidecode(data.iloc[i,0]).replace('_',' ')
    
    

    #remove special characters

In [47]:
data = data.reset_index(drop=True)

In [48]:
data.iloc[65]

Player           Jayson Tatum
G                          66
Date      2023-03-17 00:00:00
Age                    25-014
Tm                        BOS
                            @
Opp                       POR
                      W (+14)
GS                          1
MP                      36:29
FG                          8
FGA                        20
FG%                      .400
3P                          2
3PA                        11
3P%                      .182
FT                         16
FTA                        18
FT%                      .889
ORB                         1
DRB                        11
TRB                        12
AST                         4
STL                         3
BLK                         0
TOV                         5
PF                          2
PTS                        34
GmSc                     26.4
+/-                       +11
Name: 65, dtype: object

In [49]:
data['Player'][2]

'Jayson Tatum'

In [50]:
len(data['Player'].unique())

520

In [51]:
info = pd.read_csv('player_index.csv')
for i in range(len(info)):
    info.iloc[i,1] = unidecode(info.iloc[i,1]).replace('_',' ')
print(data['Player'][2])

p2pos = {player:pos for player,pos in zip(info['Player'].to_list(),info['Pos'].to_list())}
pos = []
for i in range(len(data)):
    pos.append(p2pos[data.iloc[i][0]])
pos

data['Pos'] = pos
data

Jayson Tatum


,Player,G,Date,Age,Tm,,Opp,,GS,MP,...,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-,Pos
0,Jayson Tatum,1,2022-10-18,24-229,BOS,,PHI,W (+9),1,38:38,...,12,4,1,1,3,1,35,30.1,+10,PF
1,Jayson Tatum,2,2022-10-21,24-232,BOS,@,MIA,W (+7),1,36:47,...,5,4,1,2,2,1,29,22.7,-1,PF
2,Jayson Tatum,3,2022-10-22,24-233,BOS,@,ORL,W (+6),1,37:39,...,8,1,1,2,1,2,40,34.2,+6,PF
3,Jayson Tatum,4,2022-10-24,24-235,BOS,@,CHI,L (-18),1,35:26,...,8,5,0,0,1,2,26,20.7,-8,PF
4,Jayson Tatum,5,2022-10-28,24-239,BOS,,CLE,L (-9),1,42:11,...,7,4,1,1,4,4,32,24.1,-10,PF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25141,Marko Simonovic,3,2023-02-16,23-124,CHI,,MIL,L (-12),0,1:48,...,0,0,0,0,0,1,0,-0.4,+8,C
25142,Marko Simonovic,4,2023-03-31,23-167,CHI,@,CHO,W (+30),0,3:58,...,1,0,0,0,0,1,0,-0.4,-1,C
25143,Marko Simonovic,5,2023-04-04,23-171,CHI,,ATL,L (-18),0,2:06,...,0,0,0,0,0,0,2,1.0,0,C
25144,Marko Simonovic,6,2023-04-05,23-172,CHI,@,MIL,L (-13),0,0:39,...,0,0,0,0,0,0,0,0.0,+3,C


In [52]:
minutes =  data['MP'].to_list()

for i in range(len(minutes)):
    new = minutes[i].split(':')
    res = float(new[0])+(float(new[1])/60)
    minutes[i] = res
data['MP'] = minutes


In [53]:
data.columns = [['Player', 'G', 'Date', 'Age', 'Tm', 'H/A', 'Opp', 'W/L', 'GS', 'MP', 'FG',
       'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB',
       'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'GmSc', '+/-', 'Pos']]
home = data['H/A'].values

for i in range(len(home)):
    if home[i] == '@':
        home[i] = 0
    else:
        home[i] = 1
home[0:10]


array([[1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0]], dtype=object)

In [54]:
data['H/A'] = home
data = data.replace('CHO','CHA')
data = data.replace('PHO','PHX')
data= data.replace('BRK','BKN')

WL = data['W/L'].values.astype(str)
margin = []
for i in range(len(WL)):
    input_string = WL[i][0]
    output = re.match(r"^[WL]\s*\(([-]?\d+|\+\d+)\)", input_string).group(1)
    margin.append(output)

margin[0:10]
data['W/L']=margin
data['W/L'] = data['W/L'].astype(int)
data.fillna(5)

,Player,G,Date,Age,Tm,H/A,Opp,W/L,GS,MP,...,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-,Pos
0,Jayson Tatum,1,2022-10-18,24-229,BOS,1,PHI,9,1,38.633333,...,12,4,1,1,3,1,35,30.1,+10,PF
1,Jayson Tatum,2,2022-10-21,24-232,BOS,0,MIA,7,1,36.783333,...,5,4,1,2,2,1,29,22.7,-1,PF
2,Jayson Tatum,3,2022-10-22,24-233,BOS,0,ORL,6,1,37.650000,...,8,1,1,2,1,2,40,34.2,+6,PF
3,Jayson Tatum,4,2022-10-24,24-235,BOS,0,CHI,-18,1,35.433333,...,8,5,0,0,1,2,26,20.7,-8,PF
4,Jayson Tatum,5,2022-10-28,24-239,BOS,1,CLE,-9,1,42.183333,...,7,4,1,1,4,4,32,24.1,-10,PF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25141,Marko Simonovic,3,2023-02-16,23-124,CHI,1,MIL,-12,0,1.800000,...,0,0,0,0,0,1,0,-0.4,+8,C
25142,Marko Simonovic,4,2023-03-31,23-167,CHI,0,CHA,30,0,3.966667,...,1,0,0,0,0,1,0,-0.4,-1,C
25143,Marko Simonovic,5,2023-04-04,23-171,CHI,1,ATL,-18,0,2.100000,...,0,0,0,0,0,0,2,1.0,0,C
25144,Marko Simonovic,6,2023-04-05,23-172,CHI,0,MIL,-13,0,0.650000,...,0,0,0,0,0,0,0,0.0,+3,C


In [55]:
data['Date'].max()

Date   2023-04-05
dtype: datetime64[ns]

In [56]:
KM_vals = pd.read_csv('KM_vals.csv',index_col=0)

In [57]:
KM_dict = dict(zip(KM_vals.Player,KM_vals.KM))

In [58]:
data.Player.values[0][0]

'Jayson Tatum'

In [59]:
temp = []
for p in data.Player.values:
    if p[0] in KM_dict:
        temp.append(KM_dict[p[0]])
    else:
        temp.append(15)
data['KM'] = temp

In [ ]:
data[['FG%','3P%','FT%']] = data[['FG%','3P%','FT%']].replace(5.00,0)

In [60]:
data.to_csv('data.csv',index=False)

In [61]:
data

,Player,G,Date,Age,Tm,H/A,Opp,W/L,GS,MP,...,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-,Pos,KM
0,Jayson Tatum,1,2022-10-18,24-229,BOS,1,PHI,9,1,38.633333,...,4,1,1,3,1,35,30.1,+10,PF,7
1,Jayson Tatum,2,2022-10-21,24-232,BOS,0,MIA,7,1,36.783333,...,4,1,2,2,1,29,22.7,-1,PF,7
2,Jayson Tatum,3,2022-10-22,24-233,BOS,0,ORL,6,1,37.650000,...,1,1,2,1,2,40,34.2,+6,PF,7
3,Jayson Tatum,4,2022-10-24,24-235,BOS,0,CHI,-18,1,35.433333,...,5,0,0,1,2,26,20.7,-8,PF,7
4,Jayson Tatum,5,2022-10-28,24-239,BOS,1,CLE,-9,1,42.183333,...,4,1,1,4,4,32,24.1,-10,PF,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25141,Marko Simonovic,3,2023-02-16,23-124,CHI,1,MIL,-12,0,1.800000,...,0,0,0,0,1,0,-0.4,+8,C,15
25142,Marko Simonovic,4,2023-03-31,23-167,CHI,0,CHA,30,0,3.966667,...,0,0,0,0,1,0,-0.4,-1,C,15
25143,Marko Simonovic,5,2023-04-04,23-171,CHI,1,ATL,-18,0,2.100000,...,0,0,0,0,0,2,1.0,0,C,15
25144,Marko Simonovic,6,2023-04-05,23-172,CHI,0,MIL,-13,0,0.650000,...,0,0,0,0,0,0,0.0,+3,C,15


In [62]:
data.Date.max()

Date   2023-04-05
dtype: datetime64[ns]